# Import Movie Review Data

Set the seed

In [1]:
import numpy as np

In [2]:
np.random.seed(42)

Import the dataset as pandas dataframe

In [3]:
import pandas as pd

Data can be downloaded from Kaggle at the following URL

- https://www.kaggle.com/c/word2vec-nlp-tutorial/data

In [4]:
df = pd.read_csv('labeledTrainData.tsv.zip',header=0, delimiter="\t", quoting=3)

Split Data into Training and Test Data

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    df['review'],
    df['sentiment'],
    test_size=0.2, 
    random_state=42
)

# Build the Tokenizer

In [7]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [8]:
top_words = 10000

In [9]:
t = Tokenizer(num_words=top_words) # num_words -> Vocablury size

In [10]:
t.fit_on_texts(X_train.tolist())

# Prepare Training and Test Data

Get the word index for each of the word in the review

In [11]:
X_train = t.texts_to_sequences(X_train.tolist())

In [12]:
X_test = t.texts_to_sequences(X_test.tolist())

How many words in each review?

# Pad Sequences - Important

In [13]:
from tensorflow.python.keras.preprocessing import sequence

In [14]:
max_review_length = 300

In [15]:
X_train = sequence.pad_sequences(X_train,maxlen=max_review_length,padding='post')

In [16]:
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length, padding='post')

# Build Embedding Matrix from Pre-Trained Word2Vec

Load pre-trained Gensim Embeddings

In [17]:
import gensim

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [18]:
word2vec = gensim.models.Word2Vec.load('word2vec-movie-50')

Embedding Size

In [19]:
embedding_vector_length = word2vec.wv.syn0.shape[1]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  if __name__ == '__main__':


Build matrix for current data

In [20]:
embedding_matrix = np.zeros((top_words + 1, embedding_vector_length))

In [21]:
for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
    if i > top_words:
        break
    if word in word2vec.wv.vocab:
        embedding_vector = word2vec.wv[word]
        embedding_matrix[i] = embedding_vector

# Build the Graph

In [22]:
from tensorflow.python.keras.models import Sequential

In [23]:
from tensorflow.python.keras.layers import Dropout, Dense, Embedding, Flatten, LSTM

In [24]:
model = Sequential()

Add Embedding layer
 - Embedding Layer Input = Batch_Size * Length of each review

In [25]:
model.add(Embedding(top_words + 1,
                    embedding_vector_length,
                    input_length=max_review_length,
                   weights=[embedding_matrix],
                   trainable=False)
         )

Embedding Layer Output - 
[Batch_Size , Review Length , Embedding_Size]

Add Layer with 100 LSTM Memory Units

In [26]:
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
#outout is last hidden state
#batch_size, 100

In [27]:
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# Execute the graph

In [ ]:
model.fit(X_train,y_train,
          epochs=10,
          batch_size=128,          
          validation_data=(X_test, y_test),
         verbose=2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================]20000/20000 [==============================] - 152s 8ms/step - loss: 0.6723 - acc: 0.5645 - val_loss: 0.6471 - val_acc: 0.6694